# Use `ibm-watsonx-ai` to deploy Python script
This notebook contains steps and code to demonstrate how to deploy a Python script with the [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in the PyPI repository. This notebook consists of steps to create a pPthon script, create a deployment, create and run a job. 

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Learning goals
The learning goals of this notebook are:

- Create and save a python script.
- Deploy the script using the client library.
- Create and Run a job which utilises the created deployment.

## Contents
This notebook contains the following parts:

1. [Setup](#section1)
2. [Deployment of Python Script](#section2)
3. [Create and Run Job](#section3)
4. [Clean up](#section4)
5. [Summary](#section5)

<a id='section1'></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact your Cloud Pack for Data administrator and ask them for your account credentials

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [156]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [157]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.1"
)

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.1"
)

```

In [158]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [159]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [160]:
client.set.default_space(space_id)

'SUCCESS'

### Software specification

You can use popular tools, libraries, and frameworks to train and deploy machine learning models and functions.

The following list shows the predefined (base) model types and software specifications.

In [ ]:
client.software_specifications.list()

You can select the software specification using the function below.

In [161]:
base_sw_spec_id = client.software_specifications.get_uid_by_name("runtime-24.1-py3.11")
print(base_sw_spec_id)

45f12dfe-aa78-5b8d-9f38-0ee223c47309


<a id='section2'></a>
## 2. Python Script Deployment

### Save Python Script

This file will be saved locally so you can deploy and run it later.

In [162]:
%%writefile /tmp/CreateSwSpec_script.py

import sys
import time
import os

outfname = os.path.join(os.environ.get('BATCH_OUTPUT_DIR'), "swspec.log")
with open(outfname, "w") as f:
    try:
        f.write("The deployed python script was run successfully!")
    except Exception as ex:
        f.write("The deployed python script failed: " + repr(sys.exc_info()[0]))
        f.write("sys path:")
        f.write(sys.path)

Overwriting /tmp/CreateSwSpec_script.py


The file should be successfully created. 
To check its content, you can use the command below.

In [163]:
!cat /tmp/CreateSwSpec_script.py


import sys
import time
import os

outfname = os.path.join(os.environ.get('BATCH_OUTPUT_DIR'), "swspec.log")
with open(outfname, "w") as f:
    try:
        f.write("The deployed python script was run successfully!")
    except Exception as ex:
        f.write("The deployed python script failed: " + repr(sys.exc_info()[0]))
        f.write("sys path:")
        f.write(sys.path)


### Deployment of Python Script

You can store and deploy a Python script and get its details by running the code in following cells. 

In [164]:
meta_props = {
    client.script.ConfigurationMetaNames.NAME: "PyScript",
    client.script.ConfigurationMetaNames.SOFTWARE_SPEC_UID: base_sw_spec_id  # required
}

In [137]:
script_path = "/tmp/CreateSwSpec_script.py"
script_details = client.script.store(meta_props, file_path=script_path)
script_id = client.script.get_id(script_details)
print("Created script ", script_id)

Creating Script asset...
SUCCESS
Created script  55b4015a-b8af-424c-bbe7-9010b682d93d


In [ ]:
script_details

In [165]:
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "pyscript_deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S' }
}

deployment_details = client.deployments.create(script_id, deployment_meta_props)
deployment_id = client.deployments.get_id(deployment_details)



######################################################################################

Synchronous deployment creation for id: '55b4015a-b8af-424c-bbe7-9010b682d93d' started

######################################################################################


ready.


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='8a771b18-f88c-4fa6-acaf-2425b5a2c046'
-----------------------------------------------------------------------------------------------




In [ ]:
deployment_details

<a id='section3'></a>
## 3. Create and Run job

Run the following cells to create and run a job with the deployed script.

In [166]:
def poll_async_job(wml_client, job_uid):
    import time
    while True:
        job_status = wml_client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status['state']
        if state == 'completed' or 'fail' in state:
            return wml_client.deployments.get_job_details(job_uid)
        time.sleep(5)

In [168]:
job_payload_ref = {
    client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
        'type': 'data_asset',
        'location': { 'name': 'deploy_test_script-out' }
    }
}

job = client.deployments.create_job(deployment_id, meta_props=job_payload_ref)
job_id = client.deployments.get_job_id(job)

In [169]:
job_details = poll_async_job(client, job_id)

{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '2025-01-07T11:25:42.988Z', 'state': 'running'}
{'completed_at': '2025-01-07T11:25:47.868622Z', 'running_at': '2025-01-07T11:25:42.956402Z', 'state': 'completed'}


In [ ]:
client.data_assets.list()

To download the asset run the code below. It will be downloaded as a zip archive.

In [ ]:
scoring_params = client.deployments.get_job_details(job_id)

from ibm_watsonx_ai.helpers import DataConnection
output_data_connection = DataConnection.from_dict(scoring_params["entity"]["scoring"]["output_data_reference"])
output_data_connection.set_client(client)

output_data_connection.download("/tmp/CreateSwSpec_script_result.zip")

In [173]:
import zipfile

with zipfile.ZipFile("/tmp/CreateSwSpec_script_result.zip", "r") as zip_ref:
    zip_ref.extractall("/tmp/script_result_extracted_files")

zip_ref.namelist()

['swspec.log']

In [178]:
with open("/tmp/script_result_extracted_files/swspec.log", "r") as file:
    file_content = file.readlines()

file_content

['The deployed python script was run successfully!']

<a id='section4'></a>
## 4. Cleanup 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id='section5'></a>
## 5. Summary

You successfully completed this notebook! You learned how to create and deploy a python script, and create and run a job using Watson Machine Learning.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 


### Author

**Jakub Żywiecki**, Software Engineer

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.